### MY470 Computer Programming

### Week 4 Assignment, MT 2021

#### \*\*\* Due 12:00 noon on Monday, October 25 \*\*\*

---
### Writing your own k-means clustering algorithm

K-means clustering is a simple unsupervised machine-learning method for cluster analysis. The aim of the method is to partition a set of points into k clusters, such that each point is assigned to the nearest cluster. The algorithm iterates through two steps:

1. Assign each data point to the cluster with the nearest centroid
2. Update the centroids of the clusters given the new assignment

The algorithm converges when the assignments no longer change. Since the intial assignment to clusters is largely random, there is no guarantee that the optimum assignment is found. So it is common to run the algorithm multiple times and use different starting conditions.

In this assignment, we will implement a much simplified version of the k-means clustering algorithm. Rather than running the algorithm until convergence, we will repeat the above two steps a large but fixed number of times. In addition, we will initialize only once, using a naive method according to which we randomly choose k points from the data to use as initial cluster centroids. 

(In real life, you will of course use a library to implement such an algorithm. In Python, you can do this using [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).)

For the assignment, we will additionally use data from the file "Wholesale customers data.csv". The file contains information on the annual spending on diverse product categories for the clients of a wholesale distributor. The data are obtained from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php) and you can find more information about them [here](http://archive.ics.uci.edu/ml/datasets/Wholesale+customers#).

#### Hints

Use docstrings to describe your functions. We will subtract points from your mark if you do not use appropriate description of your code.

In [4]:
# We will first import the modules we need
# Edit this cell if you prefer to use alternative modules or libraries

# You will need the math module to estimate the square root.
# To get the square root of num, use math.sqrt(num)
import math
import csv
import random

### Problem 1: Function to estimate Euclidean distance between two points

Write a function called `get_distance` that calculates the Euclidean distance between two n-dimensional points. The function should take two lists as arguments, where each list contains the n coordinates of each of the two points. 

Test your function for the points [0, 3, 0] and [4, 0, 0].

#### Hints

You can read about the definition of Euclidean distance on [Wikipedia](https://en.wikipedia.org/wiki/Euclidean_distance).


In [112]:
# Enter your answer to Problem 1 below. 

def get_distance(x, y):
    """Assumes x and y are numeric type. The aim of the function is to find the distance between n-dimensional points."""
    """Function returns nothing if the lenght of x an y are different so that we make sure they have the same dimensions."""  
    if len(x) != len(y):
        return None
    
    dist = 0
    for i in range(0,len(x)):
        dist = dist + (x[i]-y[i])**2
    
    return math.sqrt(dist)
        
print(get_distance([0, 3, 0],[4, 0, 0])) 


5.0


### Problem 2: Function to estimate the centroid of a collection of points

Write a function called `get_centroid` that estimates the centroid of a collection of n-dimensional points. The function should take one list as an argument, which contains each of the points entered as a list of n coordinates. The function should return a list with the coordinates of the virtual center point.

Test your function for the points in `test_lst` entered below.

#### Hints

The coordinate of the centroid in each dimension is the mean of the coordinates of all the points in that dimension.


In [113]:
test_lst = [[0,0,0], [0,0,1], [0,1,0], [1,0,0], 
            [0,1,1], [1,0,1], [1,1,0], [1,1,1]]

# Enter your answer to Problem 2 below. 

def get_centroid(z):
    """Assumes z is of numeric type. The aim of the function is to find the centroid of various n-dimensional points."""
    """The function calculates the mean of coordinates. It takes a list of coordinates and returns a list with the coordinates of the centroid."""
    centroid = []
    for i in range(0, len(z[0])):
        total = 0
        for j in range(0, len(z)):
            total = total + (z[j][i])
    
        centroid.append(total/len(z)) 
    return centroid

print(get_centroid(test_lst))


[0.5, 0.5, 0.5]


---
### Problem 3: Function to read data

Write a function called `get_data` that opens the file "Wholesale customers data.csv" and returns all the data in a list. Each element in the list should be a list of each customer's annual spending on fresh products, milk products, grocery products, frozen products, detergents and paper products, and delicatessen products. In other words, your list should contain 440 elements (customers), each of which contains six numeric elements (amounts spent on products). The function does not need to take any arguments.

Test your function by saving the data it returns in a variable called `data`. Then print the first two elements of `data`.


#### Hints

Use the csv module to read the file. You can read how to do this [here](https://docs.python.org/3/library/csv.html). Make sure you do not include the column names in the data. 

In [115]:
# Enter your answer to Problem 3 here. 

def get_data():
    """This function first opens the csv file at hand, and then returns all the data in a list."""
    """The function ignores the column names in the file and returns customer spending in each 6 category as a numeric value."""
    with open('Wholesale customers data.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        list_of_annual_spending = []
    
        next(csv_reader)
        for line in csv_reader:
            line = [int(x) for x in line]
            list_of_annual_spending.append(line[2:8])
            
    
    return list_of_annual_spending

data = get_data()

print(data[:2])


[[12669, 9656, 7561, 214, 2674, 1338], [7057, 9810, 9568, 1762, 3293, 1776]]


---
### Problem 4: Function to implement k-means algorithm

Write a function called `kmeans` that clusters a collection of points into k clusters using a simplified version of the k-means algorithm. The function should take two arguments: 

1. `points` – a list of n-dimensional points, and
2. `k` – an integer that defines the number of desired clusters. 

The function should return two things: 

1. A clustering – a list of `k` clusters, each of which is a list of points (each of which is a list of coordinates)
2. A list of the centroids for each of the `k` clusters. Each centroid is essentially a point, so it should be presented as a list of coordinates.

Write your code around the detailed comments and the helping code below.

Test your function on the data from Problem 3 for k = 3. For each of the three clusters, print the number of customers assigned to it and the cordinates of its centroid.


In [121]:
# Enter your answer to Problem 4 in-between the code and comments below. 

def kmeans(points, k):
    """This function clusters a collection of points into k clusters. It first assigns each point to the cluster with the closest centroid."""
    """Then the function goes onto create a new clustering with new centroids based on the previous clusters' centroids."""
    """The function repeats the process described above 100 times in order to refine the clustering."""
    """The function returns a list of specified numbers of clusters and a list of centroids for each of those clusters."""
    
    # Select k random points to use as initial centroids
    init = random.sample(points, k)

    # Create a list of k lists to contain the points assigned to each cluster.  
    clusters = [[] for i in init]
    
    # Create a list to keep the centroids of the k clusters. 
    # For now, this list will contain the points from init.
    centroids = [i for i in init]
    
    # You now need to assign each point to the cluster 
    # with the closest centroid. Use the get_distance function 
    # you wrote in Problem 1 for this.
    for i in points:
        closest_distance = float('inf')
        closest_centroid = -1
        for j in centroids:
            if closest_distance > get_distance(i, j):
                closest_distance = get_distance(i, j)
                closest_centroid = centroids.index(j)
        clusters[closest_centroid].append(i)
   
    
    # You should then update the variable "clusters" to be 
    # the new clustering and update the variable "centroids" 
    # to contain the centroids of the clusters in this new clustering.
    # Use the function you wrote in Problem 2 to estimate the centroids.
    
    # Wrote the described process here first and ran it for 100 times in the following loop below
    
    #for i in range(0, k):
        #centroids[i] = get_centroid(clusters[i])

    #clusters = [[] for i in init]
    #for i in points:
        #closest_distance = float('inf') 
        #closest_centroid = -1
        #for j in centroids:
            #if closest_distance > get_distance(i, j):
                #closest_distance = get_distance(i, j)
                #closest_centroid = centroids.index(j)
        #clusters[closest_centroid].append(i)
    
        
    # Repeat the process described above for 100 iterations. 
    # The idea is that each new repetition refines the clustering 
    # because it starts from the centroids of the previous clustering. 
    # If we repeat the process long enough, the assignment to 
    # clusters and the centroids will become stable.
    
    for i in range(0 , 100):
        for i in range(0, k):
            centroids[i] = get_centroid(clusters[i])

        clusters = [[] for i in init]
        for i in points:
            closest_distance = float('inf')
            closest_centroid = -1
            for j in centroids:
                if closest_distance > get_distance(i, j):
                    closest_distance = get_distance(i, j)
                    closest_centroid = centroids.index(j)
            clusters[closest_centroid].append(i)

    return clusters, centroids 


x = kmeans(data, 3)

cluster_length = []
for i in x[0]:
    cluster_length.append(len(i))

for i in range(0, len(x[1])):
    print('Cluster ', i + 1, ': Number of Customers :', cluster_length[i], 'Centroid Coordinates :', x[1][i])
    

Cluster  1 : Number of Customers : 337 Centroid Coordinates : [7390.958456973293, 4439.768545994065, 6292.19584569733, 2495.53412462908, 2238.652818991098, 1158.448071216617]
Cluster  2 : Number of Customers : 75 Centroid Coordinates : [32768.013333333336, 4827.68, 5723.1466666666665, 5535.92, 1074.12, 2066.64]
Cluster  3 : Number of Customers : 28 Centroid Coordinates : [11849.17857142857, 24717.10714285714, 33887.71428571428, 3409.3214285714284, 15459.714285714286, 4483.857142857143]


---

### Evaluation

| Problem | Mark     | Comment   
|:-------:|:--------:|:----------------------
| 1       |   /2    |              
| 2       |   /2    | 
| 3       |   /2    | 
| 4       |   /6    | 
| Legibility      |   /2    | 
| Modularity      |   /2    | 
| Efficiency      |   /4    | 
|**Total**|**/20**  | 